### Data Quality Analysis

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go

In [ ]:
def get_all_symbols():
    """ Get all symbols in database as a DataFrame """
    conn = sqlite3.connect("TSX_Prices.sqlite")
    sql = f"SELECT * FROM symbols ORDER BY ticker"
    data = pd.read_sql_query(sql, conn, index_col="ticker")
    data.drop(labels=["index", "url", "yahoo"], axis=1, inplace=True )

    return data

In [ ]:
    # data.index = pd.to_datetime(data.index, format="%Y-%m-%d", errors='coerce')
    # DataFrame.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')


In [ ]:
df = get_all_symbols()
df

In [140]:
def get_all_prices():
    conn = sqlite3.connect("TSX_Prices.sqlite")
    sql = f"SELECT * FROM prices_dailt ORDER BY UPPER(ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="ticker")
    
    #data.drop(labels=["index", "url", "yahoo"], axis=1, inplace=True )

    return prices

In [165]:
conn = sqlite3.connect("TSX_Prices.sqlite")
sql = f"SELECT * FROM prices_daily ORDER BY Ticker ASC, Date ASC"
prices = pd.read_sql_query(sql, conn, index_col="Date")
prices.drop("index", axis=1, inplace=True)
prices


,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2021-11-04 00:00:00,A,-,-,-,0.55,0.0
2021-11-05 00:00:00,A,0.61,0.61,0.61,0.61,1500.0
2021-11-08 00:00:00,A,-,-,-,0.61,100.0
2021-11-09 00:00:00,A,-,-,-,0.61,0.0
2021-11-10 00:00:00,A,0.62,0.62,0.56,0.56,1013.0
...,...,...,...,...,...,...
2021-11-24 00:00:00,test,0.12,0.135,0.12,0.12,177376.0
2021-11-25 00:00:00,test,0.14,0.14,0.11,0.11,268727.0
2021-11-26 00:00:00,test,0.11,0.175,0.11,0.15,948075.0


In [ ]:
prices["new_ticker"] = np.where(prices["Ticker"] != prices["Ticker"].shift(1), "New", "")
prices["cur_date"]  = pd.to_datetime(prices.index, format="%Y-%m-%d", errors='coerce')
prices["prev_date"] = pd.to_datetime(np.where(prices["new_ticker"] != "New", prices["cur_date"].shift(1), None), format="%Y-%m-%d", errors='coerce')

prices["GAP"] = prices["cur_date"] - prices["prev_date"]
prices["missing"] = prices["GAP"] > datetime.timedelta(days=5)



#filter = prices["new_ticker"] == "New"
#prices.loc[filter]
# df["Trend"] = np.where(df["Close"] > df["SMA200"], "Up", "Down")

In [174]:
filter = (prices["missing"] == True) & (prices["Ticker"] == "SHOP")
prices.loc[filter]

,Ticker,Open,High,Low,Close,Volume,new_ticker,cur_date,prev_date,GAP,missing
Date,,,,,,,,,,,
2015-11-02 00:00:00,SHOP,42.36,43.99,40.52,40.52,51399.0,,2015-11-02,2015-10-02,31 days,True
2017-11-27 00:00:00,SHOP,145.37,148.97,144.05,144.69,444455.0,,2017-11-27,2017-10-27,31 days,True
2021-10-27 00:00:00,SHOP,1708.38,1711.11,1652.36,1681.67,152957.0,,2021-10-27,2021-09-29,28 days,True
